In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip

In [ ]:
!unzip ./val2017.zip

In [ ]:
!git clone https://github.com/martinber/dl_football_detection.git

In [ ]:
%cd dl_football_detection

In [ ]:
import train
import tensorflow as tf
from pathlib import Path

CASES_PATH = Path("./cases/saved/")
VAL2017_FOLDER_PATH = Path("../val2017")

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print(
        '\n\nThis error most likely means that this notebook is not '
        'configured to use a GPU.  Change this in Notebook Settings via the '
        'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise SystemError('GPU device not found')
else:
    print("GPU detected")

print("Tensorflow version:", tf.__version__)

In [ ]:
metrics = [
    tf.keras.metrics.BinaryCrossentropy(from_logits=False),
    tf.keras.metrics.BinaryAccuracy(threshold=0.5),
    tf.keras.metrics.Precision(thresholds=0.5),
    tf.keras.metrics.Recall(thresholds=0.5),
    tf.keras.metrics.MeanAbsoluteError(),
    tf.keras.metrics.MeanSquaredError(),
]

for n_filters in [4, 8, 16]:

    model = train.u_net(input_size=(None,None,3), n_filters=n_filters, n_classes=1)

    case = train.Case(
          model=model,
          batch_size=16,
          num_batches=100,
          num_epochs=100,
          optimizer=tf.keras.optimizers.Adam(lr=1e-3),
          loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=metrics,
          gen_params={
              # Dataset: Background and ball images
              "folder_path": str(VAL2017_FOLDER_PATH),
              "obj_path": str(Path("./res/ball_2_transparent.png")),

              # Shape of object in ground truth, "rect" or "ellipse"
              "ground_truth_shape": "ellipse",

              # Needed divisibility of the width and height of images. Depends
              # in amount of downsampling
              "divisibility": 32,

              # Size of images, make divisible by previous parameter or
              # otherwise padding will be added.
              # Used in training dataset but also in validation dataset during
              # training, but not during evaluation.
              "train_val_img_size": (128, 128),

              # Modifications to object
              "ball_rotate": True,
              "ball_size_range": (20, 70),
              "ball_brightness": (0.4, 1),
          },
          notes="",
      )


    with tf.device('/device:GPU:0'):
        train.train_case(case, CASES_PATH)


In [ ]:
!zip -r cases.zip ./cases/
# After this you can download the cases.zip with the models, and then evaluate
# them with `./main.py eval` in your PC